<center>

# Домашнее задание по Статистическим и Эмпирическим Методам Компьютинга

### Подготовил Куприянов Кирилл Игоревич, студент гр. БПИ 151(1)

</center> Задание 4

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import linear_model
import scipy.stats as sts
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Часть 1

In [ ]:
df = pd.read_excel('Данные к задаче 4 дз (мультиколлинеарность).xls')

x2, x3, x4, y = list(df['x2_7']), list(df['x3_7']), list(df['x4_7']), list(df['y_7'])

## Регрессионный анализ методом наименьших квадратов

In [ ]:
def reg_m(y, x):
    x = np.array(x).T
    x = sm.add_constant(x)
    results = sm.OLS(endog=y, exog=x).fit()
    return results

In [ ]:
reg = reg_m(y, [x2, x3, x4])
print(reg.summary())

### Проверка значимости регресии в целом
Гипотеза $H_0$: Все 3 коэффициента (при x2, x3 и x4) нули

Альтернативная гипотеза $H_A$: Хотя бы 1 из 3 коэффициентов не равен 0

In [ ]:
print('statistic: {}'.format(reg.fvalue))
print('p-value: {}'.format(reg.f_pvalue))

Если $pvalue$ $\leqslant$ $\alpha$ для табличных значений распределения Фишера $F(q, n-q-1, \alpha)$, то $H_0$ отвергается

$q$ - число ограниченй = 3

$n$ - число точек в выборке = 40

$H_0$ отвергается

### Проверка гипотезы о совместной значимости коэффициентов при переменных x3 и x4

Для каждой объясняющей переменной (x2, x3, x4):

Нулевая гипотеза $H_0$: $x_i$ = 0
Альтернативная гипотеза $H_A$: $x_i$ != 0

In [ ]:
for i in range(1, 4):
    print('x{}:'.format(str(i+1)))
    print('\t t-statistic = {}'.format(str(reg.tvalues[i])))
    print('\t pvalue = {}'.format(str(reg.pvalues[i])))

Если $pvalue \leqslant \alpha$ для табличных значений $t$-распределения $t(a, n - m -1)$, то $H_0$ отвергается.

$n$ - кол-во наблюдений

$m$ - кол-во объясняющих переменных, включенных в регрессию

2.031. Все $pvalue$ объясняющих переменных меньше, значит все коэффициенты значимы

### Проверка гипотезы о совместной значимости коэффициентов при переменных $x_3$ и $x_4$
Гипотеза $H_0$: $B_3$ = $B_4$ = 0

Альтернативная гипотеза $H_A$: NOT $H_0$: $(B_3^2 + B_4^2 > 0)$

In [ ]:
reg2 = reg_m(y, [x2])
R2_ur = reg.rsquared
R2_r = reg2.rsquared
((R2_ur - R2_r) / 2) / ((1 - R2_ur) / (len(y) - 4))

In [ ]:
len(y)-4

F(2, 36) = 5.25 ($\alpha$ = 0.01) = Fcrit

F-val=6.10 > Fcrit=5.25  ->  $H_0$ отвергается и коэффициенты при $x_3$ и $x_4$ совместно значимы

### Корреляционная матрица для объясняющих переменных


In [ ]:
print(np.corrcoef([x2, x3, x4]))

# Часть 2
### График зависимости оценок коэффициентов от alpha

In [ ]:
def graph(alphas):
    n_alphas = len(alphas)
    clf = linear_model.Ridge(fit_intercept=False)

    coefs = []
    for a in alphas:
        clf.set_params(alpha=a)
        clf.fit(np.array([x2, x3, x4]).T, y)
        coefs.append(clf.coef_)
    
    ax = plt.gca()

    ax.plot(alphas, coefs)
    ax.set_xscale('log')
    #ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
    plt.xlabel('alpha')
    plt.ylabel('estimation')
    plt.title('Ridge coefficients as a function of the regularization')
    plt.axis('tight')
    plt.show()

In [ ]:
graph(np.arange(0., 100.1, .1))
graph(np.arange(0., 100000., 10))

С увеличением параметра alpha оценка коэффициентов стремится к 0